# Log File Parsing Demonstration

This Jupyter Notebook shows how to parse a log file to extract key fields such as IP address, timestamp, request method, URL, status code, response size, and query parameters.

Note: This notebook is for demonstration part of the parsing part only. The full project runs in a `ETL.py` file.

# Step 1: Read the file


    I checked the log file in Notepad and it had 1,129 lines.
    When we read the file, it should have the same number of lines.

In [138]:
path = r"DATA/nginx_logs - Copy.txt"
with open(path, "r") as f:
    logs = f.readlines()
    logs = [log.strip() for log in logs] # This is for triming and getting rid of those '\n's.
    print("number of logs:\t",len(logs))

number of logs:	 1129


# Step 2: Parse it

    Now we have a list of the nginx logs.lets take a look at it.

In [139]:
logs[:9]

['74.240.191.221 - - [25/Jun/2024:00:06:22 +0000] "DELETE /index.html?product_id=585&user_id=218 HTTP/1.1" 200 1974',
 '15.89.35.67 - - [24/Jun/2024:04:43:22 +0000] "PUT /index.html?session_id=def456&session_id=mno345&user_id=709 HTTP/1.1" 301 4349',
 '158.20.204.46 - - [21/Jun/2024:14:54:22 +0000] "GET /contact.html?session_id=def456 HTTP/1.1" 200 4060',
 '158.20.204.46 - - [21/Jun/2024:14:54:22 +0000] "GET /contact.html?session_id=def456 HTTP/1.1" 200 4060',
 '238.189.173.32 - - [21/Jun/2024:21:33:22 +0000] "GET /signup?session_id=abc123&session_id=ghi789 HTTP/1.1" 500 2661',
 '190.111.164.7 - - [26/Jun/2024:09:03:22 +0000] "GET /products.html?session_id=mno345&user_id=483 HTTP/1.1" 302 629',
 '196.224.137.79 - - [22/Jun/2024:09:44:22 +0000] "POST /login?session_id=ghi789 HTTP/1.1" 200 2661',
 '248.187.87.40 - - [21/Jun/2024:16:46:22 +0000] "DELETE /signup?session_id=ghi789 HTTP/1.1" 200 734',
 '158.20.204.46 - - [24/Jun/2024:06:15:22 +0000] "DELETE /products.html?session_id=mno345&p

- For ease of use and clarity of my explanation, I will demonstrate the process on an element of this list and then apply it to the entire dataset.

In [140]:
log = logs[0]
log

'74.240.191.221 - - [25/Jun/2024:00:06:22 +0000] "DELETE /index.html?product_id=585&user_id=218 HTTP/1.1" 200 1974'

In [141]:
from datetime import datetime

In [142]:
def parse_logfile(logfile):
    """
    Parses a log entry, extracts the necessary fields, and returns a dictionary.
    
    Parameters:
    logfile (str): A single log entry string.
    
    Returns:
    dict: A dictionary containing the extracted fields.
    """
    o = {}
    
    first_split = logfile.split(" - - ") 
    ip_address = first_split[0].strip()
    
    second_split = first_split[1].replace("[","").split("]")
    timestamp = second_split[0].strip()
    
    http_req, url, _, status_code, response_size = second_split[1].replace("\"","").split()
    query_param = url.split("?")[1] if "?" in url else ""
        
    o["IP_Address"] = ip_address
    o["Timestamp"] = datetime.strptime(timestamp,"%d/%b/%Y:%H:%M:%S %z")
    o["Request_Method"] = http_req   
    o["URL"] = url
    o["Status_Code"] = int(status_code)
    o["Query_Parameters"] = query_param
    
    # Some of resopnse sizes are "-" and not int type
    # o["Response_Size"] = response_size if type(response_size) == int else 0
    o["Response_Size"] = 0 if response_size == "-" else response_size

    return o

- Here is the output of the function:

In [143]:
output = parse_logfile(log)
output

{'IP_Address': '74.240.191.221',
 'Timestamp': datetime.datetime(2024, 6, 25, 0, 6, 22, tzinfo=datetime.timezone.utc),
 'Request_Method': 'DELETE',
 'URL': '/index.html?product_id=585&user_id=218',
 'Status_Code': 200,
 'Query_Parameters': 'product_id=585&user_id=218',
 'Response_Size': '1974'}

- Now I want to do these processes for the more logs.

In [144]:
output = []

for log in logs:
    o = parse_logfile(log)
    output.append(o)

In [145]:
import pandas as pd

df = pd.DataFrame(output)
df

,IP_Address,Timestamp,Request_Method,URL,Status_Code,Query_Parameters,Response_Size
0,74.240.191.221,2024-06-25 00:06:22+00:00,DELETE,/index.html?product_id=585&user_id=218,200,product_id=585&user_id=218,1974
1,15.89.35.67,2024-06-24 04:43:22+00:00,PUT,/index.html?session_id=def456&session_id=mno34...,301,session_id=def456&session_id=mno345&user_id=709,4349
2,158.20.204.46,2024-06-21 14:54:22+00:00,GET,/contact.html?session_id=def456,200,session_id=def456,4060
3,158.20.204.46,2024-06-21 14:54:22+00:00,GET,/contact.html?session_id=def456,200,session_id=def456,4060
4,238.189.173.32,2024-06-21 21:33:22+00:00,GET,/signup?session_id=abc123&session_id=ghi789,500,session_id=abc123&session_id=ghi789,2661
...,...,...,...,...,...,...,...
1124,147.77.212.177,2024-06-23 20:12:22+00:00,POST,/login?product_id=15&product_id=900,500,product_id=15&product_id=900,4049
1125,13.224.24.185,2024-06-25 22:48:22+00:00,DELETE,/services.html?product_id=6&session_id=abc123,301,product_id=6&session_id=abc123,1500
1126,35.239.32.150,2024-06-20 08:28:22+00:00,POST,/index.html?product_id=170,200,product_id=170,4292
1127,151.80.55.55,2024-06-19 20:48:22+00:00,DELETE,/index.html?user_id=834,301,user_id=834,3508


# Step 3: Clean it
    Now let's see if the df has any duplicates or not.

In [146]:
len(df[df.duplicated()])

129

In [147]:
df[df.duplicated()]

,IP_Address,Timestamp,Request_Method,URL,Status_Code,Query_Parameters,Response_Size
3,158.20.204.46,2024-06-21 14:54:22+00:00,GET,/contact.html?session_id=def456,200,session_id=def456,4060
9,158.20.204.46,2024-06-24 06:15:22+00:00,DELETE,/products.html?session_id=mno345&product_id=16,302,session_id=mno345&product_id=16,3432
21,134.207.168.109,2024-06-20 12:18:22+00:00,PUT,/signup?category=home,200,category=home,1500
24,57.179.44.69,2024-06-19 23:09:22+00:00,GET,/products.html?session_id=ghi789&product_id=78...,500,session_id=ghi789&product_id=783&user_id=239,4292
28,127.132.184.75,2024-06-22 21:17:22+00:00,DELETE,/login?user_id=465&product_id=648,302,user_id=465&product_id=648,404
...,...,...,...,...,...,...,...
1077,NULL,2024-06-21 22:41:22+00:00,PUT,/contact.html?product_id=398,200,product_id=398,734
1082,62.82.168.113,2024-06-20 01:14:22+00:00,POST,/index.html?product_id=241&session_id=def456,500,product_id=241&session_id=def456,3405
1101,224.15.134.151,2024-06-23 03:22:22+00:00,PUT,/contact.html?user_id=784&product_id=583&produ...,200,user_id=784&product_id=583&product_id=798,2810
1114,250.58.233.174,2024-06-20 12:05:22+00:00,POST,/login?product_id=50&user_id=345&product_id=543,302,product_id=50&user_id=345&product_id=543,2517


In [148]:
df = df.drop_duplicates()
df

,IP_Address,Timestamp,Request_Method,URL,Status_Code,Query_Parameters,Response_Size
0,74.240.191.221,2024-06-25 00:06:22+00:00,DELETE,/index.html?product_id=585&user_id=218,200,product_id=585&user_id=218,1974
1,15.89.35.67,2024-06-24 04:43:22+00:00,PUT,/index.html?session_id=def456&session_id=mno34...,301,session_id=def456&session_id=mno345&user_id=709,4349
2,158.20.204.46,2024-06-21 14:54:22+00:00,GET,/contact.html?session_id=def456,200,session_id=def456,4060
4,238.189.173.32,2024-06-21 21:33:22+00:00,GET,/signup?session_id=abc123&session_id=ghi789,500,session_id=abc123&session_id=ghi789,2661
5,190.111.164.7,2024-06-26 09:03:22+00:00,GET,/products.html?session_id=mno345&user_id=483,302,session_id=mno345&user_id=483,629
...,...,...,...,...,...,...,...
1124,147.77.212.177,2024-06-23 20:12:22+00:00,POST,/login?product_id=15&product_id=900,500,product_id=15&product_id=900,4049
1125,13.224.24.185,2024-06-25 22:48:22+00:00,DELETE,/services.html?product_id=6&session_id=abc123,301,product_id=6&session_id=abc123,1500
1126,35.239.32.150,2024-06-20 08:28:22+00:00,POST,/index.html?product_id=170,200,product_id=170,4292
1127,151.80.55.55,2024-06-19 20:48:22+00:00,DELETE,/index.html?user_id=834,301,user_id=834,3508


- Now I save the df in a csv file just in case.

In [149]:
df.to_csv(r"nginx_logs.csv")

PermissionError: [Errno 13] Permission denied: 'nginx_logs.csv'

In [ ]:
# This was just for analyzing
df[(df['Request_Method']=="GET") & (df["Status_Code"] == 200)]

,IP_Address,Timestamp,Request_Method,URL,Status_Code,Query_Parameters,Response_Size
2,158.20.204.46,2024-06-21 14:54:22+00:00,GET,/contact.html?session_id=def456,200,session_id=def456,4060
13,147.77.212.177,2024-06-23 14:28:22+00:00,GET,/index.html?product_id=50&session_id=mno345&se...,200,product_id=50&session_id=mno345&session_id=jkl012,3981
18,167.53.66.101,2024-06-24 05:05:22+00:00,GET,/login?session_id=ghi789&user_id=806&session_i...,200,session_id=ghi789&user_id=806&session_id=ghi789,3414
34,153.6.128.136,2024-06-21 13:17:22+00:00,GET,/products.html?product_id=89,200,product_id=89,2287
52,86.238.237.49,2024-06-20 02:02:22+00:00,GET,/faq.html?session_id=mno345&category=home,200,session_id=mno345&category=home,847
64,190.111.164.7,2024-06-20 19:21:22+00:00,GET,/about.html?category=fashion,200,category=fashion,2120
107,238.189.173.32,2024-06-24 10:12:22+00:00,GET,/login?user_id=258,200,user_id=258,4349
114,160.229.223.17,2024-06-25 03:59:22+00:00,GET,/contact.html?user_id=186,200,user_id=186,596
126,93.228.77.196,2024-06-22 00:27:22+00:00,GET,/login?session_id=jkl012&user_id=366,200,session_id=jkl012&user_id=366,734
189,63.113.225.225,2024-06-21 05:04:22+00:00,GET,/faq.html?product_id=948,200,product_id=948,2053
